In [1]:
import pandas as pd
import re 
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
np.random.seed(12345)

In [2]:
notclean = pd.read_csv('cleanprep.csv', delimiter=',', error_bad_lines=False,engine = 'python',header = None)

In [3]:
notclean.head()

,0,1,2,3,4
0,2018-07-11 19:35:15.363270,b'tj',"b""Next two weeks prob v boring (climb up to 9k...",0.007273,0.590909
1,2018-07-11 19:35:15.736769,b'Kool_Kheart',b'@Miss_rinola But you\xe2\x80\x99ve heard abo...,0.000000,0.000000
2,2018-07-11 19:35:15.744769,b'Gary Lang',b'Duplicate skilled traders automatically with...,0.625000,0.500000
3,2018-07-11 19:35:15.867339,b'Jobs in Fintech',b'Project Manager - Technical - FinTech - Cent...,0.000000,0.175000
4,2018-07-11 19:35:16.021448,b'ERC20',"b'Coinbase App Downloads Drop, Crypto Hype Fad...",0.333333,0.500000


In [4]:
notclean.columns =['dt', 'name','text','polarity','sensitivity']
notclean =notclean.drop(['name','text'], axis=1)
notclean.head()

,dt,polarity,sensitivity
0,2018-07-11 19:35:15.363270,0.007273,0.590909
1,2018-07-11 19:35:15.736769,0.000000,0.000000
2,2018-07-11 19:35:15.744769,0.625000,0.500000
3,2018-07-11 19:35:15.867339,0.000000,0.175000
4,2018-07-11 19:35:16.021448,0.333333,0.500000


In [5]:
notclean['dt'] = pd.to_datetime(notclean['dt'])
notclean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413001 entries, 0 to 1413000
Data columns (total 3 columns):
dt             1413001 non-null datetime64[ns]
polarity       1413001 non-null float64
sensitivity    1413001 non-null float64
dtypes: datetime64[ns](1), float64(2)
memory usage: 32.3 MB


In [6]:
notclean['DateTime'] = notclean['dt'].dt.floor('h')
notclean.head()

,dt,polarity,sensitivity,DateTime
0,2018-07-11 19:35:15.363270,0.007273,0.590909,2018-07-11 19:00:00
1,2018-07-11 19:35:15.736769,0.000000,0.000000,2018-07-11 19:00:00
2,2018-07-11 19:35:15.744769,0.625000,0.500000,2018-07-11 19:00:00
3,2018-07-11 19:35:15.867339,0.000000,0.175000,2018-07-11 19:00:00
4,2018-07-11 19:35:16.021448,0.333333,0.500000,2018-07-11 19:00:00


In [7]:
vdf = notclean.groupby(pd.Grouper(key='dt',freq='H')).size().reset_index(name='tweet_vol') # to find tweet volume
vdf.head()
vdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 2 columns):
dt           302 non-null datetime64[ns]
tweet_vol    302 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 4.8 KB


In [8]:
vdf.index = pd.to_datetime(vdf.index)
vdf=vdf.set_index('dt')
vdf.head()

,tweet_vol
dt,
2018-07-11 19:00:00,1747
2018-07-11 20:00:00,4354
2018-07-11 21:00:00,4432
2018-07-11 22:00:00,3980
2018-07-11 23:00:00,3830


In [9]:
notclean.index = pd.to_datetime(notclean.index)
notclean.head()

,dt,polarity,sensitivity,DateTime
1970-01-01 00:00:00.000000000,2018-07-11 19:35:15.363270,0.007273,0.590909,2018-07-11 19:00:00
1970-01-01 00:00:00.000000001,2018-07-11 19:35:15.736769,0.000000,0.000000,2018-07-11 19:00:00
1970-01-01 00:00:00.000000002,2018-07-11 19:35:15.744769,0.625000,0.500000,2018-07-11 19:00:00
1970-01-01 00:00:00.000000003,2018-07-11 19:35:15.867339,0.000000,0.175000,2018-07-11 19:00:00
1970-01-01 00:00:00.000000004,2018-07-11 19:35:16.021448,0.333333,0.500000,2018-07-11 19:00:00


In [10]:
vdf['tweet_vol'] =vdf['tweet_vol'].astype(float)
vdf.head()

,tweet_vol
dt,
2018-07-11 19:00:00,1747.0
2018-07-11 20:00:00,4354.0
2018-07-11 21:00:00,4432.0
2018-07-11 22:00:00,3980.0
2018-07-11 23:00:00,3830.0


In [11]:
df = notclean.groupby('DateTime').agg(lambda x: x.mean()) #hourly mean of polarity and sensitivity

In [12]:
df.head()

,polarity,sensitivity
DateTime,,
2018-07-11 19:00:00,0.109705,0.187557
2018-07-11 20:00:00,0.102657,0.216148
2018-07-11 21:00:00,0.098004,0.218612
2018-07-11 22:00:00,0.096688,0.231342
2018-07-11 23:00:00,0.103997,0.217739


In [13]:
df['Tweet_vol'] = vdf['tweet_vol']
df = df.drop(df.index[0])
df.head()

,polarity,sensitivity,Tweet_vol
DateTime,,,
2018-07-11 20:00:00,0.102657,0.216148,4354.0
2018-07-11 21:00:00,0.098004,0.218612,4432.0
2018-07-11 22:00:00,0.096688,0.231342,3980.0
2018-07-11 23:00:00,0.103997,0.217739,3830.0
2018-07-12 00:00:00,0.094383,0.195256,3998.0


In [14]:
df.tail()

,polarity,sensitivity,Tweet_vol
DateTime,,,
2018-07-24 04:00:00,0.121358,0.236000,4475.0
2018-07-24 05:00:00,0.095163,0.216924,4808.0
2018-07-24 06:00:00,0.088992,0.220173,6036.0
2018-07-24 07:00:00,0.091439,0.198279,6047.0
2018-07-24 08:00:00,0.071268,0.218217,2444.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 301 entries, 2018-07-11 20:00:00 to 2018-07-24 08:00:00
Data columns (total 3 columns):
polarity       301 non-null float64
sensitivity    301 non-null float64
Tweet_vol      301 non-null float64
dtypes: float64(3)
memory usage: 9.4 KB


In [16]:
btcDF = pd.read_csv('btcprice.csv', error_bad_lines=False,engine = 'python') #load bitcoin data
btcDF.head()

,Timestamp,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
0,2018-07-10 01:00,6666.75,6683.90,6635.59,6669.73,281.73,1875693.72,6657.70
1,2018-07-10 02:00,6662.44,6674.60,6647.00,6647.00,174.10,1160103.29,6663.38
2,2018-07-10 03:00,6652.52,6662.82,6621.99,6632.53,231.41,1536936.22,6641.70
3,2018-07-10 04:00,6631.17,6655.48,6625.54,6635.92,120.38,799154.77,6638.52
4,2018-07-10 05:00,6632.81,6651.06,6627.64,6640.57,94.00,624289.31,6641.32


In [17]:
btcDF['Timestamp'] = pd.to_datetime(btcDF['Timestamp'])
btcDF = btcDF.set_index(pd.DatetimeIndex(btcDF['Timestamp']))
btcDF = btcDF.drop(['Timestamp'], axis=1)
btcDF.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Timestamp,,,,,,,
2018-07-10 01:00:00,6666.75,6683.90,6635.59,6669.73,281.73,1875693.72,6657.70
2018-07-10 02:00:00,6662.44,6674.60,6647.00,6647.00,174.10,1160103.29,6663.38
2018-07-10 03:00:00,6652.52,6662.82,6621.99,6632.53,231.41,1536936.22,6641.70
2018-07-10 04:00:00,6631.17,6655.48,6625.54,6635.92,120.38,799154.77,6638.52
2018-07-10 05:00:00,6632.81,6651.06,6627.64,6640.57,94.00,624289.31,6641.32


In [18]:
btcDF = btcDF.drop(['Weighted Price'],axis=1 )
btcDF.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency)
Timestamp,,,,,,
2018-07-10 01:00:00,6666.75,6683.90,6635.59,6669.73,281.73,1875693.72
2018-07-10 02:00:00,6662.44,6674.60,6647.00,6647.00,174.10,1160103.29
2018-07-10 03:00:00,6652.52,6662.82,6621.99,6632.53,231.41,1536936.22
2018-07-10 04:00:00,6631.17,6655.48,6625.54,6635.92,120.38,799154.77
2018-07-10 05:00:00,6632.81,6651.06,6627.64,6640.57,94.00,624289.31


In [19]:
Final_df = pd.merge(df,btcDF, how='inner',left_index=True, right_index=True)

In [20]:
Final_df.head()

,polarity,sensitivity,Tweet_vol,Open,High,Low,Close,Volume (BTC),Volume (Currency)
2018-07-11 20:00:00,0.102657,0.216148,4354.0,6342.97,6354.19,6291.00,6350.00,986.73,6231532.37
2018-07-11 21:00:00,0.098004,0.218612,4432.0,6352.99,6370.00,6345.76,6356.48,126.46,804221.55
2018-07-11 22:00:00,0.096688,0.231342,3980.0,6350.85,6378.47,6345.00,6361.93,259.10,1646353.87
2018-07-11 23:00:00,0.103997,0.217739,3830.0,6362.36,6381.25,6356.74,6368.78,81.54,519278.69
2018-07-12 00:00:00,0.094383,0.195256,3998.0,6369.49,6381.25,6361.83,6380.00,124.55,793560.22


In [21]:
Final_df.tail()

,polarity,sensitivity,Tweet_vol,Open,High,Low,Close,Volume (BTC),Volume (Currency)
2018-07-23 21:00:00,0.107282,0.235636,5164.0,7746.99,7763.59,7690.16,7706.00,237.63,1836633.86
2018-07-23 22:00:00,0.094493,0.271796,4646.0,7699.13,7759.99,7690.50,7750.09,63.31,489000.25
2018-07-23 23:00:00,0.074246,0.231640,4455.0,7754.57,7777.00,7715.45,7722.32,280.46,2173424.81
2018-07-24 00:00:00,0.080870,0.219367,3862.0,7722.95,7730.61,7690.17,7719.62,496.48,3830571.66
2018-07-24 01:00:00,0.090717,0.212626,4620.0,7712.46,7727.70,7691.14,7723.22,163.99,1264085.79


In [22]:
Final_df.columns = ['Polarity', 'Sensitivity','Tweet_vol','Open','High','Low', 'Close_Price', 'Volume_BTC', 'Volume_Dollar']
Final_df.head()

,Polarity,Sensitivity,Tweet_vol,Open,High,Low,Close_Price,Volume_BTC,Volume_Dollar
2018-07-11 20:00:00,0.102657,0.216148,4354.0,6342.97,6354.19,6291.00,6350.00,986.73,6231532.37
2018-07-11 21:00:00,0.098004,0.218612,4432.0,6352.99,6370.00,6345.76,6356.48,126.46,804221.55
2018-07-11 22:00:00,0.096688,0.231342,3980.0,6350.85,6378.47,6345.00,6361.93,259.10,1646353.87
2018-07-11 23:00:00,0.103997,0.217739,3830.0,6362.36,6381.25,6356.74,6368.78,81.54,519278.69
2018-07-12 00:00:00,0.094383,0.195256,3998.0,6369.49,6381.25,6361.83,6380.00,124.55,793560.22


In [32]:
Final_df.to_csv("Final_Data.csv", sep=',', encoding='utf-8', index=True)